# Making a mock observation from the EAGLE simulations

This script is an example script to lay out the individual steps of making mock observations.

The steps are as follows:

1. Load the EAGLE simulated data.  Specify the distance at which we want to observe the cosmic web i.e. redshift of central wavelength of filter and the resolution that the data should be binned to.  The minimum resolution at 50Mpc away is about 14".
2. Cut out a chunk of the simulated data that corresponds to the Dragonfly FOV.  (switched with step 3 because takes too long for the whole simulation)
3. Add the noise to the simulated data to mimic observational noises (includes sky background noise, read out noise, dark current noise, and shot noise).  This depends on the exposure time of the mock observation.
4. Plot the data to check and see what it looks like.

Ta da!

In [7]:
import numpy as np
import eagle_constants_and_units as c
import cosmo_utils as csu
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import mpl_toolkits.axes_grid1 as axgrid
from astropy import constants as const
from astropy import units as u

import os
import get_halpha_SB

%matplotlib inline

%run 'load_data.ipynb'
%run 'add_noise.ipynb'
%run 'extract_FOV.ipynb'

In [8]:
machine='chinook'

resolution = 14.; slicewidth = 20; exptime = 1000.*60.*60.; CMOS = False; ROnoise= None

for distance in ['500Mpc','200Mpc','100Mpc','50Mpc']:
    print "****** Working on mock observation for distance %s ******"%distance
    print 'loading the data...'
    data_tuple = loaddata(machine=machine,resolution=resolution,distance=distance,slicewidth=slicewidth)
    print 'extracting the FOV...'
    data_FOV, xystarts, size = extractFOV(data_tuple,resolution,distance)
    
    print 'adding noise to the data...'
    noiseadded_signal,B_sky_array,R_array = addnoise(data_FOV,resolution,exptime=exptime,\
                                                     CMOS=CMOS,R=ROnoise,debugging=True)
    
    print 'saving the noise-added data...'
    if CMOS:
        cams = 'newcam'
    else:
        cams = 'oldcam'
    if ROnoise:
        cams = cams+'_ROnoise%s'%ROnoise

    fname = 'noisy_fov_%shr_%s_%sdist_%sarcsec_%sslwd.npz' \
            % (round(exptime/3600.),cams,distance,resolution,slicewidth)
    R_fname = 'noisy_fov_%shr_%s_%sdist_%sarcsec_%sslwd_RARRAY.npz' \
            % (round(exptime/3600.),cams,distance,resolution,slicewidth)
    B_sky_fname = 'noisy_fov_%shr_%s_%sdist_%sarcsec_%sslwd_BSKYARRAY.npz' \
            % (round(exptime/3600.),cams,distance,resolution,slicewidth)

    print "Saving mock observation as %s"%fname
    np.savez(fname,noiseadded_signal)
    print "xystarts are:"
    print xystarts
    print "size is:"
    print size
    print "Saving readout noise array as %s"%R_fname
    np.savez(R_fname,R_array)
    print "Saving the sky background array as %s"%B_sky_fname
    np.savez(B_sky_fname,B_sky_array)
    
    print "****** Done mock observation for distance %s ******"%distance
    print ""

****** Working on mock observation for distance 500Mpc ******
loading the data...
Loading data of slicewidth 20 (after rounding), number of slices is 4
data_500Mpc_14arcsec_20slwd.npz exists, loading now...
extracting the FOV...
adding noise to the data...
******* Adding noise to the input data to simulate a mock observation by the 48.0 lens Dragonfly Telescope *******
DEBUGGING: the binpix_size (resolution) is 14.0
DEBUGGING: the pixel size (inherent) is 2.8
VERBOSE: Using old cameras... (QE = 0.48, R = 10.)
DEBUGGING: R is : 10.0
DEBUGGING: the number of pixels per bin is 25.0
DEBUGGING: raise the data by 10** since was logged data before...
DEBUGGING: the total object signal [electrons] detected ranges from: 2.3979e-16 to 858139.0
The width of the filter that the data was taken with is 3.0 nm. 
DEBUGGING: Sky background in filter width of 3.0 nm is calculated already: 1.473626.
Adding sky background noise and shot noise...
DEBUGGING: the background in the bandwidth is: 1.473626 phot

In [9]:
noiseadded_signal.shape

(829, 553)